In [22]:
#parsing of Matthew in order to make a proof of concept for a graded level reader in this book
import pandas as pd
import numpy as np
from re import search 

In [34]:
#Get data set into the python
mattToJohn = pd.read_excel('MattToJohn.xlsx')
gospels = np.array(mattToJohn)

In [37]:
mounceWordFrequency = pd.read_excel('MounceFrequencyGreek.xlsx')
freq = np.array(mounceWordFrequency)

In [33]:
#Contains the (verse: conjugation: count) to be used for weights using the gospels data
for row in gospels:
    if (search("41_Mat", str(row[0]))):
        print(row[0] + ":  " + row[2] + " : " + row[6])
     

41_Mat.001.001:  Βίβλος : βίβλος
41_Mat.001.001:  γενέσεως : γένεσις
41_Mat.001.001:  Ἰησοῦ : Ἰησοῦς
41_Mat.001.001:  Χριστοῦ : Χριστός
41_Mat.001.001:  υἱοῦ : υἱός
41_Mat.001.001:  Δαυὶδ : Δαυείδ, Δαυίδ, Δαβίδ
41_Mat.001.001:  υἱοῦ : υἱός
41_Mat.001.001:  Ἀβραάμ. : Ἀβραάμ
41_Mat.001.002:  Ἀβραὰμ : Ἀβραάμ
41_Mat.001.002:  ἐγέννησεν : γεννάω
41_Mat.001.002:  τὸν : ὁ
41_Mat.001.002:  Ἰσαάκ· : Ἰσαάκ
41_Mat.001.002:  Ἰσαὰκ : Ἰσαάκ
41_Mat.001.002:  δὲ : δέ
41_Mat.001.002:  ἐγέννησεν : γεννάω
41_Mat.001.002:  τὸν : ὁ
41_Mat.001.002:  Ἰακώβ· : Ἰακώβ
41_Mat.001.002:  Ἰακὼβ : Ἰακώβ
41_Mat.001.002:  δὲ : δέ
41_Mat.001.002:  ἐγέννησεν : γεννάω
41_Mat.001.002:  τὸν : ὁ
41_Mat.001.002:  Ἰούδαν : Ἰούδας
41_Mat.001.002:  καὶ : καί
41_Mat.001.002:  τοὺς : ὁ
41_Mat.001.002:  ἀδελφοὺς : ἀδελφός
41_Mat.001.002:  αὐτοῦ· : αὐτός
41_Mat.001.003:  Ἰούδας : Ἰούδας
41_Mat.001.003:  δὲ : δέ
41_Mat.001.003:  ἐγέννησεν : γεννάω
41_Mat.001.003:  τὸν : ὁ
41_Mat.001.003:  Φάρες : Φάρες
41_Mat.001.003:  καὶ : καί
41_

41_Mat.005.013:  ἅλας : ἅλας
41_Mat.005.013:  τῆς : ὁ
41_Mat.005.013:  γῆς· : γῆ
41_Mat.005.013:  ἐὰν : ἐάν
41_Mat.005.013:  δὲ : δέ
41_Mat.005.013:  τὸ : ὁ
41_Mat.005.013:  ἅλας : ἅλας
41_Mat.005.013:  μωρανθῇ, : μωραίνω
41_Mat.005.013:  ἐν : ἐν
41_Mat.005.013:  τίνι : τίς
41_Mat.005.013:  ἁλισθήσεται; : ἁλίζω
41_Mat.005.013:  εἰς : εἰς
41_Mat.005.013:  οὐδὲν : οὐδείς
41_Mat.005.013:  ἰσχύει : ἰσχύω
41_Mat.005.013:  ἔτι : ἔτι
41_Mat.005.013:  εἰ : εἰ
41_Mat.005.013:  μὴ : μή
41_Mat.005.013:  βληθὲν : βάλλω
41_Mat.005.013:  ἔξω : ἔξω
41_Mat.005.013:  καὶ : καί
41_Mat.005.013:  καταπατεῖσθαι : καταπατέω
41_Mat.005.013:  ὑπὸ : ὑπό
41_Mat.005.013:  τῶν : ὁ
41_Mat.005.013:  ἀνθρώπων.¶ : ἄνθρωπος
41_Mat.005.014:  Ὑμεῖς : σύ
41_Mat.005.014:  ἐστε : εἰμί
41_Mat.005.014:  τὸ : ὁ
41_Mat.005.014:  φῶς : φῶς
41_Mat.005.014:  τοῦ : ὁ
41_Mat.005.014:  κόσμου· : κόσμος
41_Mat.005.014:  οὐ : οὐ
41_Mat.005.014:  δύναται : δύναμαι
41_Mat.005.014:  πόλις : πόλις
41_Mat.005.014:  κρυβῆναι : κρύπτω
41_Ma

41_Mat.007.019:  βάλλεται. : βάλλω
41_Mat.007.020:  ἄρα : ἄρα
41_Mat.007.020:  γε : γέ
41_Mat.007.020:  ἀπὸ : ἀπό
41_Mat.007.020:  τῶν : ὁ
41_Mat.007.020:  καρπῶν : καρπός
41_Mat.007.020:  αὐτῶν : αὐτός
41_Mat.007.020:  ἐπιγνώσεσθε : ἐπιγινώσκω
41_Mat.007.020:  αὐτούς.¶ : αὐτός
41_Mat.007.021:  Οὐ : οὐ
41_Mat.007.021:  πᾶς : πᾶς
41_Mat.007.021:  ὁ : ὁ
41_Mat.007.021:  λέγων : λέγω
41_Mat.007.021:  μοι· : ἐγώ
41_Mat.007.021:  κύριε : κύριος
41_Mat.007.021:  κύριε, : κύριος
41_Mat.007.021:  εἰσελεύσεται : εἰσέρχομαι
41_Mat.007.021:  εἰς : εἰς
41_Mat.007.021:  τὴν : ὁ
41_Mat.007.021:  βασιλείαν : βασιλεία
41_Mat.007.021:  τῶν : ὁ
41_Mat.007.021:  οὐρανῶν, : οὐρανός
41_Mat.007.021:  ἀλλ᾽ : ἀλλά
41_Mat.007.021:  ὁ : ὁ
41_Mat.007.021:  ποιῶν : ποιέω
41_Mat.007.021:  τὸ : ὁ
41_Mat.007.021:  θέλημα : θέλημα
41_Mat.007.021:  τοῦ : ὁ
41_Mat.007.021:  πατρός : πατήρ
41_Mat.007.021:  μου : ἐγώ
41_Mat.007.021:  τοῦ : ὁ
41_Mat.007.021:  ἐν : ἐν
41_Mat.007.021:  τοῖς : ὁ
41_Mat.007.021:  οὐρανοῖς. : 

41_Mat.010.037:  μου : ἐγώ
41_Mat.010.037:  ἄξιος· : ἄξιος
41_Mat.010.038:  καὶ : καί
41_Mat.010.038:  ὃς : ὅς, ἥ
41_Mat.010.038:  οὐ : οὐ
41_Mat.010.038:  λαμβάνει : λαμβάνω
41_Mat.010.038:  τὸν : ὁ
41_Mat.010.038:  σταυρὸν : σταυρός
41_Mat.010.038:  αὐτοῦ : αὐτός
41_Mat.010.038:  καὶ : καί
41_Mat.010.038:  ἀκολουθεῖ : ἀκολουθέω
41_Mat.010.038:  ὀπίσω : ὀπίσω
41_Mat.010.038:  μου, : ἐγώ
41_Mat.010.038:  οὐκ : οὐ
41_Mat.010.038:  ἔστιν : εἰμί
41_Mat.010.038:  μου : ἐγώ
41_Mat.010.038:  ἄξιος. : ἄξιος
41_Mat.010.039:  ὁ : ὁ
41_Mat.010.039:  εὑρὼν : εὑρίσκω
41_Mat.010.039:  τὴν : ὁ
41_Mat.010.039:  ψυχὴν : ψυχή
41_Mat.010.039:  αὐτοῦ : αὐτός
41_Mat.010.039:  ἀπολέσει : ἀπολλύω
41_Mat.010.039:  αὐτὴν : αὐτός
41_Mat.010.039:  καὶ : καί
41_Mat.010.039:  ὁ : ὁ
41_Mat.010.039:  ἀπολέσας : ἀπολλύω
41_Mat.010.039:  τὴν : ὁ
41_Mat.010.039:  ψυχὴν : ψυχή
41_Mat.010.039:  αὐτοῦ : αὐτός
41_Mat.010.039:  ἕνεκεν : ἕνεκα, εἵνεκεν
41_Mat.010.039:  ἐμοῦ : ἐγώ
41_Mat.010.039:  εὑρήσει : εὑρίσκω
41_Mat.01

41_Mat.013.031:  ἀγρῷ : ἀγρός
41_Mat.013.031:  αὐτοῦ· : αὐτός
41_Mat.013.032:  ὃ : ὅς, ἥ
41_Mat.013.032:  μικρότερον : μικρός
41_Mat.013.032:  μέν : μέν
41_Mat.013.032:  ἐστιν : εἰμί
41_Mat.013.032:  πάντων : πᾶς
41_Mat.013.032:  τῶν : ὁ
41_Mat.013.032:  σπερμάτων, : σπέρμα
41_Mat.013.032:  ὅταν : ὅταν
41_Mat.013.032:  δὲ : δέ
41_Mat.013.032:  αὐξηθῇ, : αὐξάνω
41_Mat.013.032:  μεῖζον : μέγας
41_Mat.013.032:  τῶν : ὁ
41_Mat.013.032:  λαχάνων : λάχανον
41_Mat.013.032:  ἐστὶν : εἰμί
41_Mat.013.032:  καὶ : καί
41_Mat.013.032:  γίνεται : γίνομαι
41_Mat.013.032:  δένδρον : δένδρον
41_Mat.013.032:  ὥστε : ὥστε
41_Mat.013.032:  ἐλθεῖν : ἔρχομαι
41_Mat.013.032:  τὰ : ὁ
41_Mat.013.032:  πετεινὰ : πετεινός
41_Mat.013.032:  τοῦ : ὁ
41_Mat.013.032:  οὐρανοῦ : οὐρανός
41_Mat.013.032:  καὶ : καί
41_Mat.013.032:  κατασκηνοῦν : κατασκηνόω
41_Mat.013.032:  ἐν : ἐν
41_Mat.013.032:  τοῖς : ὁ
41_Mat.013.032:  κλάδοις : κλάδος
41_Mat.013.032:  αὐτοῦ.¶ : αὐτός
41_Mat.013.033:  Ἄλλην : ἄλλος
41_Mat.013.033: 

...

41_Mat.018.017:  ὥσπερ : ὥσπερ
41_Mat.018.017:  ὁ : ὁ
41_Mat.018.017:  ἐθνικὸς : ἐθνικός
41_Mat.018.017:  καὶ : καί
41_Mat.018.017:  ὁ : ὁ
41_Mat.018.017:  τελώνης.¶ : τελώνης
41_Mat.018.018:  Ἀμὴν : ἀμήν
41_Mat.018.018:  λέγω : λέγω
41_Mat.018.018:  ὑμῖν· : σύ
41_Mat.018.018:  ὅσα : ὅσος
41_Mat.018.018:  ἐὰν : ἐάν
41_Mat.018.018:  δήσητε : δέω
41_Mat.018.018:  ἐπὶ : ἐπί
41_Mat.018.018:  τῆς : ὁ
41_Mat.018.018:  γῆς, : γῆ
41_Mat.018.018:  ἔσται : εἰμί
41_Mat.018.018:  δεδεμένα : δέω
41_Mat.018.018:  ἐν : ἐν
41_Mat.018.018:  τῷ : ὁ
41_Mat.018.018:  οὐρανῷ· : οὐρανός
41_Mat.018.018:  καὶ : καί
41_Mat.018.018:  ὅσα : ὅσος
41_Mat.018.018:  ἐὰν : ἐάν
41_Mat.018.018:  λύσητε : λύω
41_Mat.018.018:  ἐπὶ : ἐπί
41_Mat.018.018:  τῆς : ὁ
41_Mat.018.018:  γῆς, : γῆ
41_Mat.018.018:  ἔσται : εἰμί
41_Mat.018.018:  λελυμένα : λύω
41_Mat.018.018:  ἐν : ἐν
41_Mat.018.018:  τῷ : ὁ
41_Mat.018.018:  οὐρανῷ.¶ : οὐρανός
41_Mat.018.019:  Πάλιν : πάλιν
41_Mat.018.019:  ἀμὴν : ἀμήν
41_Mat.018.019:  λέγω : λέγω
4

41_Mat.021.011:  ὄχλοι : ὄχλος
41_Mat.021.011:  ἔλεγον· : λέγω
41_Mat.021.011:  οὗτός : οὗτος
41_Mat.021.011:  ἐστιν : εἰμί
41_Mat.021.011:  ὁ : ὁ
41_Mat.021.011:  προφήτης : προφήτης
41_Mat.021.011:  Ἰησοῦς, : Ἰησοῦς
41_Mat.021.011:  ὁ : ὁ
41_Mat.021.011:  ἀπὸ : ἀπό
41_Mat.021.011:  Ναζαρὲθ : Ναζαρέθ, Ναζαρά
41_Mat.021.011:  τῆς : ὁ
41_Mat.021.011:  Γαλιλαίας.¶ : Γαλιλαία
41_Mat.021.012:  Καὶ : καί
41_Mat.021.012:  εἰσῆλθεν : εἰσέρχομαι
41_Mat.021.012:  ὁ : ὁ
41_Mat.021.012:  Ἰησοῦς : Ἰησοῦς
41_Mat.021.012:  εἰς : εἰς
41_Mat.021.012:  τὸ : ὁ
41_Mat.021.012:  ἱερὸν : ἱερόν
41_Mat.021.012:  τοῦ : ὁ
41_Mat.021.012:  θεοῦ : θεός
41_Mat.021.012:  καὶ : καί
41_Mat.021.012:  ἐξέβαλεν : ἐκβάλλω
41_Mat.021.012:  πάντας : πᾶς
41_Mat.021.012:  τοὺς : ὁ
41_Mat.021.012:  πωλοῦντας : πωλέω
41_Mat.021.012:  καὶ : καί
41_Mat.021.012:  ἀγοράζοντας : ἀγοράζω
41_Mat.021.012:  ἐν : ἐν
41_Mat.021.012:  τῷ : ὁ
41_Mat.021.012:  ἱερῷ, : ἱερόν
41_Mat.021.012:  καὶ : καί
41_Mat.021.012:  τὰς : ὁ
41_Mat.021.012

41_Mat.023.018:  τῷ : ὁ
41_Mat.023.018:  δώρῳ : δῶρον
41_Mat.023.018:  τῷ : ὁ
41_Mat.023.018:  ἐπάνω : ἐπάνω
41_Mat.023.018:  αὐτοῦ, : αὐτός
41_Mat.023.018:  ὀφείλει. : ὀφείλω
41_Mat.023.019:  μωροὶ : μωρός
41_Mat.023.019:  καὶ : καί
41_Mat.023.019:  τυφλοί, : τυφλός
41_Mat.023.019:  τί : τίς
41_Mat.023.019:  γὰρ : γάρ
41_Mat.023.019:  μεῖζον, : μέγας
41_Mat.023.019:  τὸ : ὁ
41_Mat.023.019:  δῶρον : δῶρον
41_Mat.023.019:  ἢ : ἤ
41_Mat.023.019:  τὸ : ὁ
41_Mat.023.019:  θυσιαστήριον : θυσιαστήριον
41_Mat.023.019:  τὸ : ὁ
41_Mat.023.019:  ἁγιάζον : ἁγιάζω
41_Mat.023.019:  τὸ : ὁ
41_Mat.023.019:  δῶρον; : δῶρον
41_Mat.023.020:  ὁ : ὁ
41_Mat.023.020:  οὖν : οὖν
41_Mat.023.020:  ὀμόσας : ὄμνυμι
41_Mat.023.020:  ἐν : ἐν
41_Mat.023.020:  τῷ : ὁ
41_Mat.023.020:  θυσιαστηρίῳ : θυσιαστήριον
41_Mat.023.020:  ὀμνύει : ὄμνυμι
41_Mat.023.020:  ἐν : ἐν
41_Mat.023.020:  αὐτῷ : αὐτός
41_Mat.023.020:  καὶ : καί
41_Mat.023.020:  ἐν : ἐν
41_Mat.023.020:  πᾶσιν : πᾶς
41_Mat.023.020:  τοῖς : ὁ
41_Mat.023.020

41_Mat.025.045:  οὐδὲ : οὐδέ
41_Mat.025.045:  ἐμοὶ : ἐγώ
41_Mat.025.045:  ἐποιήσατε. : ποιέω
41_Mat.025.046:  καὶ : καί
41_Mat.025.046:  ἀπελεύσονται : ἀπέρχομαι
41_Mat.025.046:  οὗτοι : οὗτος
41_Mat.025.046:  εἰς : εἰς
41_Mat.025.046:  κόλασιν : κόλασις
41_Mat.025.046:  αἰώνιον, : αἰώνιος
41_Mat.025.046:  οἱ : ὁ
41_Mat.025.046:  δὲ : δέ
41_Mat.025.046:  δίκαιοι : δίκαιος
41_Mat.025.046:  εἰς : εἰς
41_Mat.025.046:  ζωὴν : ζωή
41_Mat.025.046:  αἰώνιον.¶ : αἰώνιος
41_Mat.026.001:  Καὶ : καί
41_Mat.026.001:  ἐγένετο : γίνομαι
41_Mat.026.001:  ὅτε : ὅτε
41_Mat.026.001:  ἐτέλεσεν : τελέω
41_Mat.026.001:  ὁ : ὁ
41_Mat.026.001:  Ἰησοῦς : Ἰησοῦς
41_Mat.026.001:  πάντας : πᾶς
41_Mat.026.001:  τοὺς : ὁ
41_Mat.026.001:  λόγους : λόγος
41_Mat.026.001:  τούτους, : οὗτος
41_Mat.026.001:  εἶπεν : ἔπω, ἐρῶ, εἶπον
41_Mat.026.001:  τοῖς : ὁ
41_Mat.026.001:  μαθηταῖς : μαθητής
41_Mat.026.001:  αὐτοῦ· : αὐτός
41_Mat.026.002:  οἴδατε : εἴδω
41_Mat.026.002:  ὅτι : ὅτι
41_Mat.026.002:  μετὰ : μετά
41_Mat.026

41_Mat.027.016:  λεγόμενον : λέγω
41_Mat.027.016:  Ἰησοῦν : Ἰησοῦς
41_Mat.027.016:  Βαραββᾶν. : Βαραββᾶς
41_Mat.027.017:  συνηγμένων : συνάγω
41_Mat.027.017:  οὖν : οὖν
41_Mat.027.017:  αὐτῶν : αὐτός
41_Mat.027.017:  εἶπεν : ἔπω, ἐρῶ, εἶπον
41_Mat.027.017:  αὐτοῖς : αὐτός
41_Mat.027.017:  ὁ : ὁ
41_Mat.027.017:  Πιλᾶτος· : Πιλᾶτος
41_Mat.027.017:  τίνα : τίς
41_Mat.027.017:  θέλετε : θέλω
41_Mat.027.017:  ἀπολύσω : ἀπολύω
41_Mat.027.017:  ὑμῖν; : σύ
41_Mat.027.017:  Ἰησοῦν : Ἰησοῦς
41_Mat.027.017:  τὸν : ὁ
41_Mat.027.017:  Βαραββᾶν : Βαραββᾶς
41_Mat.027.017:  ἢ : ἤ
41_Mat.027.017:  Ἰησοῦν : Ἰησοῦς
41_Mat.027.017:  τὸν : ὁ
41_Mat.027.017:  λεγόμενον : λέγω
41_Mat.027.017:  χριστόν; : Χριστός
41_Mat.027.018:  ᾔδει : εἴδω
41_Mat.027.018:  γὰρ : γάρ
41_Mat.027.018:  ὅτι : ὅτι
41_Mat.027.018:  διὰ : διά
41_Mat.027.018:  φθόνον : φθόνος
41_Mat.027.018:  παρέδωκαν : παραδίδωμι
41_Mat.027.018:  αὐτόν.¶ : αὐτός
41_Mat.027.019:  Καθημένου : κάθημαι
41_Mat.027.019:  δὲ : δέ
41_Mat.027.019:  αὐτοῦ 

In [60]:
freqList = []
for row in freq:
    if (search("GK", str(row[0]))):
        entry = row[0].replace(u'\xa0', u' ')
        entry = entry.split("  ")
        if (len(entry) < 4):
            continue
        print(str(entry[0]) + " : " + str(entry[1]) + " : " + str(entry[3]))

print(len(freq))
#print(str(freqList) + "\n")

GK G2 | S G2 :  Ἀαρών :  5x
GK G3 | S G3 :  Ἀβαδδών :  1x
GK G4 | S G4 :  ἀβαρής :  1x
GK G5 | S G5 :  ἀββά :  3x
GK G6 | S G6 :  Ἅβελ :  4x
GK G7 | S G7 :  Ἀβιά :  3x
GK G8 | S G8 :  Ἀβιαθάρ :  1x
GK G9 | S G9 :  Ἀβιληνή :  1x
GK G10 | S G10 :  Ἀβιούδ :  2x
GK G11 | S G11 :  Ἀβραάμ :  73x
GK G12 | S G12 :  ἄβυσσος :  9x
GK G13 | S G13 :  Ἅγαβος :  2x
GK G14 | S G14 :  ἀγαθοεργέω :  2x
GK G16 | S G15 :  ἀγαθοποιέω :  9x
GK G17 | S G16 :  ἀγαθοποιία :  1x
GK G18 | S G17 :  ἀγαθοποιός :  1x
GK G19 | S G18 :  ἀγαθός :  102x
GK G20 | S G19 :  ἀγαθωσύνη :  4x
GK G21 | S G20 :  ἀγαλλίασις :  5x
GK G22 | S G21 :  ἀγαλλιάω :  11x
GK G23 | S G22 :  ἄγαμος :  4x
GK G24 | S G23 :  ἀγανακτέω :  7x
GK G25 | S G24 :  ἀγανάκτησις :  1x
GK G26 | S G25 :  ἀγαπάω :  143x
GK G27 | S G26 :  ἀγάπη :  116x
GK G28 | S G27 :  ἀγαπητός :  61x
GK G29 | S G28 :  Ἁγάρ :  2x
GK G30 | S G29 :  ἀγγαρεύω :  3x
GK G31 | S G30 :  ἀγγεῖον :  1x
GK G32 | S G31 :  ἀγγελία :  2x
GK G33 | S G32a :  ἀγγέλλω :  1x
GK G34 | S 

GK G2088 | S G1904 :  ἐπέρχομαι :  9x
GK G2089 | S G1905 :  ἐπερωτάω :  56x
GK G2090 | S G1906 :  ἐπερώτημα :  1x
GK G2091 | S G1907 :  ἐπέχω :  5x
GK G2092 | S G1908 :  ἐπηρεάζω :  2x
GK G2093 | S G1909 :  ἐπί :  890x
GK G2094 | S G1910 :  ἐπιβαίνω :  6x
GK G2095 | S G1911 :  ἐπιβάλλω :  18x
GK G2096 | S G1912 :  ἐπιβαρέω :  3x
GK G2097 | S G1913 :  ἐπιβιβάζω :  3x
GK G2098 | S G1914 :  ἐπιβλέπω :  3x
GK G2099 | S G1915 :  ἐπίβλημα :  4x
GK G2101 | S G1917 :  ἐπιβουλή :  4x
GK G2102 | S G1918 :  ἐπιγαμβρεύω :  1x
GK G2103 | S G1919 :  ἐπίγειος :  7x
GK G2104 | S G1920 :  ἐπιγίνομαι :  1x
GK G2105 | S G1921 :  ἐπιγινώσκω :  44x
GK G2106 | S G1922 :  ἐπίγνωσις :  20x
GK G2107 | S G1923 :  ἐπιγραφή :  5x
GK G2108 | S G1924 :  ἐπιγράφω :  5x
GK G2109 | S G1925 :  ἐπιδείκνυμι :  7x
GK G2110 | S G1926 :  ἐπιδέχομαι :  2x
GK G2111 | S G1927 :  ἐπιδημέω :  2x
GK G2112 | S G1928 :  ἐπιδιατάσσομαι :  1x
GK G2113 | S G1929 :  ἐπιδίδωμι :  9x
GK G2114 | S G1930 :  ἐπιδιορθόω :  1x
GK G2115 | S G1

GK G3701 | S G3471 :  μωραίνω :  4x
GK G3702 | S G3472 :  μωρία :  5x
GK G3703 | S G3473 :  μωρολογία :  1x
GK G3704 | S G3474 :  μωρός :  12x
GK G3707 | S G3475 :  Μωϋσῆς :  80x
GK G3709 | S G3476 :  Ναασσών :  3x
GK G3710 | S G3477 :  Ναγγαί :  1x
GK G3711 | S G3478 :  Ναζαρά :  2x
GK G3714 | S G3478 :  Ναζαρέθ :  6x
GK G3715 | S G3478 :  Ναζαρέτ :  4x
GK G3716 | S G3479 :  Ναζαρηνός :  6x
GK G3717 | S G3480 :  Ναζωραῖος :  13x
GK G3718 | S G3481 :  Ναθάμ :  1x
GK G3720 | S G3482 :  Ναθαναήλ :  6x
GK G3721 | S G3483 :  ναί :  33x
GK G3722 | S G3483a :  Ναιμάν :  1x
GK G3723 | S G3484 :  Ναίν :  1x
GK G3724 | S G3485 :  ναός :  45x
GK G3725 | S G3486 :  Ναούμ :  1x
GK G3726 | S G3487 :  νάρδος :  2x
GK G3727 | S G3488 :  Νάρκισσος :  1x
GK G3728 | S G3489 :  ναυαγέω :  2x
GK G3729 | S G3490 :  ναύκληρος :  1x
GK G3730 | S G3491 :  ναῦς :  1x
GK G3731 | S G3492 :  ναύτης :  3x
GK G3732 | S G3493 :  Ναχώρ :  1x
GK G3733 | S G3494 :  νεανίας :  3x
GK G3734 | S G3495 :  νεανίσκος :  11x
G